# Import
This first block is only for starting in Colabratory. Some pips need to be re-installed each time and authentication codes need to be entered from links to allow access to google drive files. The free GPU service is well worth the small inconvenience. Remove this block if not using Colabratory.

In [0]:
from google.colab import auth
auth.authenticate_user()
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
!mkdir -p drive
!google-drive-ocamlfuse drive
!pip install -q keras
!pip install numba

# Imports


In [0]:
# from datetime import datetime 
# start_real = datetime.now()
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import train_test_split
from sklearn.pipeline import FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from keras.callbacks import Callback

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dropout, Dense, concatenate, Embedding, Flatten, Activation, SpatialDropout1D
from keras.layers import Bidirectional, GRU, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.optimizers import Adam
from keras.models import Model

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import MaxPooling1D, Conv1D
from keras.callbacks import EarlyStopping, ModelCheckpoint
from nltk.tokenize import sent_tokenize, word_tokenize

import warnings
warnings.filterwarnings('ignore')
import os
os.environ['OMP_NUM_THREADS'] = '4'

import re
import math
# set seed
np.random.seed(123)

/usr/local/lib/python3.6/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


This block is to set up and/or alter GPU settings. Remove if not using GPU

In [0]:
# GPU acceleration
import os
os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=gpu,floatX=float32"
# import theano
# don't overload gpu
from keras import backend as K
config = K.tf.ConfigProto()
config.gpu_options.allow_growth = True
session = K.tf.Session(config=config)

# Read in data
change this to target where you keep your train and test data. I also use the sample_submission later on for formating the final submission so grab that too.

In [0]:
train = pd.read_csv('drive/Toxic_Comment/train.csv')
test = pd.read_csv('drive/Toxic_Comment/test.csv')

In [0]:
print(train.shape[0])
print(test.shape[0])
train[:10]

159571
153164


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0


Split the training set into a train and validation set.

In [0]:
train, valid = train_test_split(train, random_state=233, train_size=0.95)

n_train = train.shape[0]
n_test = test.shape[0]
n_valid = valid.shape[0]

print("Training on", n_train, "examples")
print("Validating on", n_valid, "examples")
print("Testing on", n_test, "examples")

Training on 151592 examples
Validating on 7979 examples
Testing on 153164 examples


Note: There are no NA values in this data, otherwise we would remove them here.

# Preprocessing the data
I've tried several string preprocessing steps but all of them actually have resulted in no improvement or a ***worse*** fit than just leaving the comment_text otherwise raw before tokenizing. Some classic preprocessing steps might end up helping a little, but pretty much all preprocessing concerns are handled by the tokenizer, vectors, or just the RNN model in general.

-Removing stopwords or stemming words just gives the RNN less to work with. If there is enough data, an RNN can extract extra meaning from having tenses and such.

-Low corrilation words,typos,ect are removed with the Max_words in the tokenizer

-Punctuation is also removed during tokenization. Keeping some did not help the model.

# Time to Tokenize
This turns the text into numerical tokens. Also note that this step also removes all punctuation, rare words, and non words from the data on it's own. Max_words is how many different words/tokens we want to have in our texts. In this case, only the 100000 most common will be concidered. This is an easy variable that can be changed up and down for a change in fit. Tweet Tokenizer also work well on this set.

In [0]:
%%time

Max_words = 100000
print("Transforming text data to sequences...")
raw_text = np.hstack([full_df.comment.str.lower()])

print("   Fitting tokenizer...")
tok_raw = Tokenizer(num_words = Max_words,
                   #charaters in this filter string are removed before tokenizing 
                    filters='.!,?"#$%&()*+-/:;<=>@[\\]^_`{|}~\t\n')
tok_raw.fit_on_texts(raw_text)

train = train['comment_text'].values
valid = valid['comment_text'].values
test = test['comment_text'].value

print("   Transforming text to sequences...")
train_tokens = tok_raw.texts_to_sequences(train)
valid_tokens = tok_raw.texts_to_sequences(valid)
test_tokens = tok_raw.texts_to_sequences(test)


Transforming text data to sequences...
   Fitting tokenizer...
   Transforming text to sequences...
CPU times: user 37.5 s, sys: 419 ms, total: 37.9 s
Wall time: 37.3 s


This sets the lengths of all the token arrays to Max_length. Token arrays that have less "words" than Max_length get padded with 0s and token arrays with more "words" get cut short. Max_length is another easy variable to change to adjust fit.

In [0]:
Max_length = 150

X_train = sequence.pad_sequences(train_tokens, maxlen=Max_length)
X_test = sequence.pad_sequences(test_tokens, maxlen=Max_length)
X_valid = sequence.pad_sequences(valid_tokens, maxlen=Max_length)

Set the Y variables.

In [0]:
Y_var = ['toxic','severe_toxic','obscene', 'threat','insult','identity_hate']

Y_train = train[Y_var].values


Y_valid = valid[Y_var].values

# Put a GLoVe on it!
This puts the glove word vectors into the dataset. The text file is big and, depending on your space, it may overload memory. It is fine in Collaboratory, with it's 12gb memory limit. However, there will likely be a memory warning that you can ignore after these blocks finish. (Takes ~3 minutes) Download the glove vectors here and path the EMBEDDING_FILE to them (other glove vectors work here too) https://www.kaggle.com/sebastianmarkow/glove.42b.300d/data 

In [0]:
EMBEDDING_FILE = 'drive/Toxic_Comment/glove.840B.300d.txt'

In [0]:
%%time
embeddings_index = {}
with open(EMBEDDING_FILE,encoding='utf8') as f:
    for line in f:
        values = line.rstrip().rsplit(' ')
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

CPU times: user 3min 40s, sys: 6.54 s, total: 3min 47s
Wall time: 4min 24s


In [0]:
%%time
Embed_size = 300
word_index = tok_raw.word_index
#prepare embedding matrix
num_words = min(Max_words, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, Embed_size))
for word, i in word_index.items():
    if i >= Max_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

CPU times: user 330 ms, sys: 29 ms, total: 359 ms
Wall time: 356 ms


# Make the RNN model
Here is the model. It is one heavy bidirectional GRU layer. Bidirectional RNNs run twice, once forward and once backwards, to effectively allow the model to look forward and backwards on the sequence to determine the output. This is followed by a 1D CNN to help look for patterns, then pooled using both Average and Max Pooling.


In [0]:
def get_model():
    model = Sequential()
    inp = Input(shape=(Max_length, ))
    x = Embedding(Max_words, Embed_size, weights=[embedding_matrix], trainable=False)(inp)
    x = SpatialDropout1D(0.2)(x)
    x = Bidirectional(GRU(128, return_sequences=True,dropout=0.1,recurrent_dropout=0.1))(x)
    x = Conv1D(64, kernel_size = 3, padding = "valid", kernel_initializer = "glorot_uniform")(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    x = concatenate([avg_pool, max_pool])
    outp = Dense(6, activation="sigmoid")(x)
    
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy',
                  optimizer=Adam(lr=0.001),
                  metrics=['accuracy'],
                 )

    return model

model = get_model()

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, 150)          0                                            
__________________________________________________________________________________________________
embedding_8 (Embedding)         (None, 150, 300)     30000000    input_8[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d_8 (SpatialDro (None, 150, 300)     0           embedding_8[0][0]                
__________________________________________________________________________________________________
bidirectional_8 (Bidirectional) (None, 150, 256)     329472      spatial_dropout1d_8[0][0]        
__________________________________________________________________________________________________
conv1d_8 (

# Callbacks 
Adds 2 callbacks to the model, checkpoint and early stopping. Checkpoint saves the weights at every epoch that improves the loss while early stopping stops the model if loss declines. I like early stopping callback methods but it is not ideal for this competition. The model should be trained right now in about 2 epochs and will clearly start overfitting after that. Maybe with a heavier model. Still, it can help.

In [0]:
file_path="weights_base.best.hdf5"
checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
early = EarlyStopping(monitor="val_loss", mode="min", patience=1)


Load any model weights here. 

In [0]:
# model.load_weights('drive/Toxic_Comment/1_epoch_base_weights.h5')

# Trian the Model
This block trains the model. Each epoch takes about 20 minutes at this setting (on Colabs anyways). Raising the batch_size will speed it up a lot, but probably lose some accuracy and maybe need more epochs. 2, maybe 3, epochs should be ideal. 

In [0]:
%%time

from numpy.random import seed
seed(123)
from tensorflow import set_random_seed
set_random_seed(123)

model_callbacks = [checkpoint, early]
model.fit(X_train, Y_train,
          batch_size=64,
          epochs=3,
          verbose=1,
          validation_data=(X_valid, Y_valid),
          callbacks = model_callbacks)

Train on 151592 samples, validate on 7979 samples
Epoch 1/4
 43200/151592 [=======>......................] - ETA: 12:27 - loss: 0.0447 - acc: 0.9829

151552/151592 [============================>.] - ETA: 0s - loss: 0.0431 - acc: 0.9834

151592/151592 [==============================] - 1063s 7ms/step - loss: 0.0431 - acc: 0.9834 - val_loss: 0.0430 - val_acc: 0.9835

Epoch 00001: val_loss improved from 0.04440 to 0.04299, saving model to weights_base.best.hdf5
Epoch 2/4
 15424/151592 [==>...........................] - ETA: 15:43 - loss: 0.0404 - acc: 0.9849

151552/151592 [============================>.] - ETA: 0s - loss: 0.0404 - acc: 0.9844

151592/151592 [==============================] - 1070s 7ms/step - loss: 0.0404 - acc: 0.9844 - val_loss: 0.0429 - val_acc: 0.9830

Epoch 00002: val_loss improved from 0.04299 to 0.04287, saving model to weights_base.best.hdf5
Epoch 3/4
 15424/151592 [==>...........................] - ETA: 15:44 - loss: 0.0370 - acc: 0.9854

151552/151592 [============================>.] - ETA: 0s - loss: 0.0381 - acc: 0.9851

151592/151592 [==============================] - 1054s 7ms/step - loss: 0.0381 - acc: 0.9851 - val_loss: 0.0444 - val_acc: 0.9822

Epoch 00003: val_loss did not improve
Epoch 4/4
 17408/151592 [==>...........................] - ETA: 15:13 - loss: 0.0340 - acc: 0.9867

KeyboardInterrupt: ignored

# Prediction
This makes the prediction on the test data. The model saves the best weights to file_path so uncomment that line to load them if your model stopped on a bad epoch.

In [0]:
%%time
# model.load_weights(file_path)
Y_test = model.predict(X_test, batch_size=256)

CPU times: user 1min 16s, sys: 10.9 s, total: 1min 27s
Wall time: 1min 25s


# Make the submission
My best single model submission was 0.9835 ROC with this notebook. That is pretty much the highest a model can get without ensembling. Based on code that was posted after the competition, the highest single model model prediction was only around ~0.9840 ROC. So this notebook was pretty close to optimal in the Toxic Comments Compo.

In [0]:
submission = pd.read_csv("drive/Toxic_Comment/sample_submission.csv")
submission[Y_var] = Y_test
submission.to_csv("drive/Toxic_Comment/Glove_RNN_Submission10.csv", index=False)


This block permenently saves the weights. This can be read in with read_text and/or just loaded into a model with model.load_weights(the file path) then can skip training or continue to train from there.

In [0]:

model.save_weights('drive/Toxic_Comment/my_model_weights.h5')